# AI Config Create - Cookbook

This cookbook contains all code from `aiconfig-create/SKILL.md` for creating AI Configs.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `ai-configs:write` permission
- Project created (run `cookbook_aiconfig_projects.ipynb` first)
- Tools created (run `cookbook_aiconfig_tools.ipynb` first) - creates `search_knowledge_base` and `get_customer_info`

In [1]:
%pip install requests python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables from .env
import os
from dotenv import load_dotenv

def find_repo_root():
    """Find repo root by looking for .git directory"""
    path = os.getcwd()
    while path != "/":
        if os.path.exists(os.path.join(path, ".git")):
            return path
        path = os.path.dirname(path)
    return os.getcwd()

REPO_ROOT = find_repo_root()
ENV_FILE = os.path.join(REPO_ROOT, ".env")
load_dotenv(ENV_FILE)
print(f"Loaded environment from: {ENV_FILE}")

Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [3]:
import requests
import os
import time

# Configuration
API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

def delete_config(config_key: str):
    """Delete an AI Config if it exists (for clean test runs)"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.delete(url, headers=headers)
    if response.status_code == 204:
        print(f"[OK] Deleted existing config: {config_key}")
        return True
    elif response.status_code == 404:
        print(f"[INFO] Config '{config_key}' does not exist (clean slate)")
        return False
    else:
        print(f"[ERROR] Failed to delete config: {response.text}")
        return False

---
## Example 1: Complete Agent Mode Config
From: `SKILL.md` lines 170-244

Creates an AI Config with a default variation in a single API call. Includes tools and custom parameters.

**Note:** Tools (`search_knowledge_base`, `get_customer_info`) must be created first via `cookbook_aiconfig_tools.ipynb`

In [4]:
def create_agent_config():
    """
    Complete example of creating an AI Config for agent mode.
    Uses single API call with defaultVariation for efficiency.
    Includes optional tools and custom parameters.
    """

    config_key = "support-agent"

    # ============================================================
    # NOTE: Tools must be created BEFORE attaching to variations
    # See `aiconfig-tools` skill for creating tools like:
    #   - search_knowledge_base
    #   - get_customer_info
    # Tools are referenced by key in the defaultVariation's "tools" array
    # ============================================================

    # ============================================================
    # Create AI Config with defaultVariation 
    # ============================================================
    config_data = {
        "key": config_key,
        "name": "Customer Support Agent",
        "mode": "agent",
        "defaultVariation": {
            "key": "default",
            "name": "Default Configuration",
            "instructions": """You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Resolve issues efficiently
- Maintain a friendly tone

Company: {{company_name}}
Priority: {{support_priority}}""",
            "messages": [],  # Empty messages array required for agent mode
            "model": {
                "name": "openai",
                "modelName": "gpt-4",
                "parameters": {
                    # Standard LLM parameters
                    "temperature": 0.7,
                    "maxTokens": 2000,

                    # OPTIONAL: Add any custom parameters your application needs
                    # These are passed directly to your app
                    "custom_param": "value",
                    "enable_feature": True,
                    "workflow_config": {
                        "max_retries": 3,
                        "timeout": 30
                    }
                }
            },
            # Tools array - references tools created via aiconfig-tools skill
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        }
    }

    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs"
    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    response = requests.post(url, headers=headers, json=config_data)

    if response.status_code != 201:
        print(f"[ERROR] Failed to create config: {response.text}")
        return None

    print(f"[OK] Created AI Config: {config_key}")
    print(f"  URL: https://app.launchdarkly.com/{PROJECT_KEY}/ai-configs/{config_key}")
    print(f"  Default variation: 'default'")
    return response.json()

# Delete existing config first for clean test run, then create
print("=== Creating Agent Mode Config ===\n")
delete_config("support-agent")
create_agent_config()

=== Creating Agent Mode Config ===

[INFO] Config 'support-agent' does not exist (clean slate)
[OK] Created AI Config: support-agent
  URL: https://app.launchdarkly.com/support-ai/ai-configs/support-agent
  Default variation: 'default'


{'_access': {'denied': [], 'allowed': []},
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs',
   'type': 'application/json'},
  'self': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
   'type': 'application/json'}},
 'createdAt': 1770145843950,
 'description': '',
 'key': 'support-agent',
 'mode': 'agent',
 'name': 'Customer Support Agent',
 'tags': [],
 'updatedAt': 1770145843950,
 'variations': [],
 'version': 1}

---
## Example 2: Complete Completion Mode Config
From: `SKILL.md` lines 246-312

Creates a completion mode config with messages array and tools.

**Note:** Uses same tools created by `cookbook_aiconfig_tools.ipynb`

In [5]:
def create_completion_config():
    """
    Complete example of creating an AI Config for completion mode.
    Uses single API call with defaultVariation.
    Includes optional tools for function calling and custom parameters.
    """

    config_key = "content-assistant"

    # ============================================================
    # NOTE: Tools must be created BEFORE attaching to variations
    # See `aiconfig-tools` skill for creating tools like:
    #   - search_knowledge_base
    #   - get_customer_info
    # Tools are referenced by key in the defaultVariation's "tools" array
    # ============================================================

    # ============================================================
    # Create AI Config with defaultVariation
    # ============================================================
    config_data = {
        "key": config_key,
        "name": "Content Generation Assistant",
        "mode": "completion",
        "defaultVariation": {
            "key": "default",
            "name": "Default Configuration",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a creative content writer for {{brand}}. Use available tools when needed."
                },
                {
                    "role": "user",
                    "content": "{{content_request}}"
                }
            ],
            "model": {
                "name": "openai",
                "modelName": "gpt-4",
                "parameters": {
                    # Standard LLM parameters
                    "temperature": 0.9,
                    "maxTokens": 2000,
                    "topP": 0.95,

                    # OPTIONAL: Custom parameters for your application
                    "style_guide": "creative",
                    "output_format": "markdown",
                    "include_sources": True,
                    "custom_settings": {
                        "tone": "friendly",
                        "audience": "general"
                    }
                }
            },
            # Tools for function calling - references tools created via aiconfig-tools skill
            "tools": [
                {"key": "search_knowledge_base", "version": 1},
                {"key": "get_customer_info", "version": 1}
            ]
        }
    }

    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs"
    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    response = requests.post(url, headers=headers, json=config_data)

    if response.status_code != 201:
        print(f"[ERROR] Failed to create config: {response.text}")
        return None

    print(f"[OK] Created AI Config: {config_key}")
    print(f"  URL: https://app.launchdarkly.com/{PROJECT_KEY}/ai-configs/{config_key}")
    print(f"  Default variation: 'default'")
    return response.json()

# Delete existing config first for clean test run, then create
print("=== Creating Completion Mode Config ===\n")
delete_config("content-assistant")
create_completion_config()

=== Creating Completion Mode Config ===

[INFO] Config 'content-assistant' does not exist (clean slate)
[OK] Created AI Config: content-assistant
  URL: https://app.launchdarkly.com/support-ai/ai-configs/content-assistant
  Default variation: 'default'


{'_access': {'denied': [], 'allowed': []},
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs',
   'type': 'application/json'},
  'self': {'href': '/api/v2/projects/support-ai/ai-configs/content-assistant',
   'type': 'application/json'}},
 'createdAt': 1770145845050,
 'description': '',
 'key': 'content-assistant',
 'mode': 'completion',
 'name': 'Content Generation Assistant',
 'tags': [],
 'updatedAt': 1770145845050,
 'variations': [],
 'version': 1}

---
## Summary

This cookbook created:
1. **support-agent** - Agent mode config with default variation
2. **content-assistant** - Completion mode config with default variation

Both configs include:
- Tools (`search_knowledge_base`, `get_customer_info`) - created by `aiconfig-tools` skill
- Custom parameters in `model.parameters`
- Template variables (`{{variable}}`)

### Key Features Demonstrated
- **Single API call with `defaultVariation`** - Efficient config creation
- **Custom parameters** - Pass any custom data to your application via `model.parameters`
- **Tool references** - Tools created separately and referenced by key

Next steps:
- Run `cookbook_aiconfig_variations.ipynb` to add more variations
- Run `cookbook_aiconfig_sdk.ipynb` to consume configs with the SDK